In [1]:
import suite2p
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
import scipy.io
import pandas as pd
import re
import os
import glob
from natsort import natsorted
from multiprocessing import Pool
from joblib import Parallel, delayed
from pathlib import Path
import gc
gc.collect()
import importlib
import sys
from pathlib import Path
# Add the directory containing your script to the Python path
from suite2p import run_s2p

sys.path.append(r'C:\Users\zhangl33\Projects\Project-SLMonlineControl\CodeAndPackages\PYsubfun')

# Now you can import your script as a module
import FastBin_Suite2p as FBS
import CalBin2p as CB
import SLM_Offline as slmO

In [2]:
data= loadmat('D:\Project1-LocalProcessing\Step1\FOV.mat')
FOV=data['FOVUpdate']
FOVsave=data['suite2pFOVPath']
FOVsave

array([[array(['\\\\nimhlabstore1.nimh.nih.gov\\UFNC\\FNC3Z\\Sutie2p-Processed\\GCamP6S-CamKII\\\\SL085503042025\\'],
              dtype='<U89')                                                                                          ,
        array(['\\\\nimhlabstore1.nimh.nih.gov\\UFNC\\FNC3Z\\Sutie2p-Processed\\GCamP6S-CamKII\\\\SL085503062025\\'],
              dtype='<U89')                                                                                          ,
        array(['\\\\nimhlabstore1.nimh.nih.gov\\UFNC\\FNC3Z\\Sutie2p-Processed\\GCamP6S-CamKII\\\\SL086404162025\\'],
              dtype='<U89')                                                                                          ,
        array(['\\\\nimhlabstore1.nimh.nih.gov\\UFNC\\FNC3Z\\Sutie2p-Processed\\GCamP6S-CamKII\\\\SL086404222025\\'],
              dtype='<U89')                                                                                          ,
        array(['\\\\nimhlabstore1.nimh.nih.gov\\UFNC

In [3]:

ConfigFolder=r'C:\Users\zhangl33\Projects\Project-SLMonlineControl\config'
ops0, confSet=FBS.configLoad(ConfigFolder,'SLMsetting.yml','ops3PlanesG6.npy')
ops0['spatial_scale']=0
ops0['threshold_scaling']=1.6
ops0['denoise']=0
ops0['tau']=1.5
ops0['look_one_level_down']=False               ##This is important as the subfolders is defined in db
ops0['combined']=True

folder = r'D:\Project1-LocalProcessing\Step1'  # your path
all_csv = [f'FOV{i}.csv' for i in range(1,FOV.shape[1]+1)]
dfs = [pd.read_csv(os.path.join(folder, f)) for f in all_csv]

In [10]:

FOVsave=data['suite2pFOVPath']
Keyword =['Spon','SLMall']
for i in range(FOV.shape[1]):  
    FOVtemp = FOV[0, i]
    FOVtempSave=FOVsave[0,i]
    dfTemp=dfs[i]
    ops0['data_path']=FOVtemp['DataFolder']
    root     = Path(ops0['data_path'][0])
    confdir=root.parent.parent.parent
    names, pairs = slmO.subfolders_and_pairs(FOVtempSave[0],Keyword)
    confSet=FBS.read_yaml(os.path.join(confdir,'CurrentSLMsetting.yml'))

    for name, pair in zip(names, pairs):
        full_save_path = os.path.join(FOVtempSave[0], name)
        suite2pdir= os.path.join(full_save_path, 'suite2p')
        ops0['save_path0']= full_save_path
        FBS.CombinePlanes(suite2pdir, ops0)


appended plane 0 to combined view
appended plane 1 to combined view
appended plane 2 to combined view
3  planes combined
Folder already exists.
appended plane 0 to combined view
appended plane 1 to combined view
appended plane 2 to combined view
3  planes combined
Folder already exists.
appended plane 0 to combined view
appended plane 1 to combined view
appended plane 2 to combined view
3  planes combined
Folder already exists.
appended plane 0 to combined view
appended plane 1 to combined view
appended plane 2 to combined view
3  planes combined
Folder already exists.
appended plane 0 to combined view
appended plane 1 to combined view
appended plane 2 to combined view
3  planes combined
Folder already exists.
appended plane 0 to combined view
appended plane 1 to combined view
appended plane 2 to combined view
3  planes combined
Folder already exists.
appended plane 0 to combined view
appended plane 1 to combined view
appended plane 2 to combined view
3  planes combined
Folder already 

In [8]:
## spontnous period
succRateTh = 0.96
FOVsave=data['suite2pFOVPath']
DistTh = 8
sucTable = []
Keyword =['Spon','SLMall']
#for i in range(FOV.shape[1]):
for i in range(2,FOV.shape[1]):  
    FOVtemp = FOV[0, i]
    FOVtempSave=FOVsave[0,i]
    dfTemp=dfs[i]

    ops0['data_path']=FOVtemp['DataFolder']
    root     = Path(ops0['data_path'][0])
    print(ops0['denoise'])
    
    confdir=root.parent.parent.parent
    FilteredPos3D, FilteredFunScore, FilteredTestPos3D, FilteredTestTBL = slmO.load_slm_data(root.parent)
    names, pairs = slmO.subfolders_and_pairs(FOVtempSave[0],Keyword)
    confSet=FBS.read_yaml(os.path.join(confdir,'CurrentSLMsetting.yml'))

    for name, pair in zip(names, pairs):

        full_save_path = os.path.join(FOVtempSave[0], name)
        suite2pdir= os.path.join(full_save_path, 'suite2p')
        print(full_save_path)
        CellPos3D, CellPos3DRaw, CaData, CaDataPlane, stat=slmO.extract_suite2p(suite2pdir, confSet)
        SLMtarget, SLMtargetCellDist=slmO.slm_target_match_cell(FilteredPos3D, CellPos3DRaw, DistTh)

        succRate=np.sum(SLMtarget>=0)/SLMtarget.shape[0]

        print(CaData['ops']['threshold_scaling'])
        if CaData['ops']['threshold_scaling']==1.8:
              
              ops0['spatial_scale'] = 0
              ops0['threshold_scaling'] = 1.6

              ops0['save_path0']= full_save_path
              ops0['fast_disk']=  ops0['save_path0']
              print(ops0['save_path0'])
              print(ops0['fast_disk'])
              ops0['combined']='False'                  ## as the data is processed previous in Local machine,

              slmO.run_s2p_by_range(root, ops0, id_range=pair)
              FBS.CombinePlanes(suite2pdir, ops0)
              #CellPos3D, CellPos3DRaw, CaData, CaDataPlane, stat=slmO.extract_suite2p(suite2pdir, confSet)


0
\\nimhlabstore1.nimh.nih.gov\UFNC\FNC3Z\Sutie2p-Processed\GCamP6S-CamKII\\SL086404162025\sub1-2Spon
1.8
\\nimhlabstore1.nimh.nih.gov\UFNC\FNC3Z\Sutie2p-Processed\GCamP6S-CamKII\\SL086404162025\sub1-2Spon
\\nimhlabstore1.nimh.nih.gov\UFNC\FNC3Z\Sutie2p-Processed\GCamP6S-CamKII\\SL086404162025\sub1-2Spon
{'data_path': ['E:\\LuSLMOnlineTest\\SL0864\\04162025\\SingleP\\Top35SpeedStimEdgeExc\\Data'], 'subfolders': ['TSeries-04162025-1028-001', 'TSeries-04162025-1028-002']}
FOUND BINARIES AND OPS IN ['\\\\nimhlabstore1.nimh.nih.gov\\UFNC\\FNC3Z\\Sutie2p-Processed\\GCamP6S-CamKII\\\\SL086404162025\\sub1-2Spon\\suite2p\\plane0\\ops.npy', '\\\\nimhlabstore1.nimh.nih.gov\\UFNC\\FNC3Z\\Sutie2p-Processed\\GCamP6S-CamKII\\\\SL086404162025\\sub1-2Spon\\suite2p\\plane1\\ops.npy', '\\\\nimhlabstore1.nimh.nih.gov\\UFNC\\FNC3Z\\Sutie2p-Processed\\GCamP6S-CamKII\\\\SL086404162025\\sub1-2Spon\\suite2p\\plane2\\ops.npy']
removing previous detection and extraction files, if present
>>>>>>>>>>>>>>>>>>>>> P

c:\users\zhangl33\projects\suite2p\suite2p\detection\sparsedetect.py:286: UserWarning: Spatial scale estimation failed.  Setting spatial scale to 1 in order to continue.
  warn(


NOTE: FORCED spatial scale ~6 pixels, time epochs 1.00, threshold 8.00 
0 ROIs, score=95.39
Detected 823 ROIs, 8.90 sec
After removing overlaps, 783 ROIs remain
----------- Total 76.66 sec.
----------- EXTRACTION
Masks created, 0.42 sec.
Extracted fluorescence from 783 ROIs in 8200 frames, 9.12 sec.
----------- Total 11.10 sec.
----------- CLASSIFICATION
['skew', 'compact', 'npix_norm']
----------- SPIKE DECONVOLUTION
----------- Total 0.27 sec.
Plane 0 processed in 159.23 sec (can open in GUI).
>>>>>>>>>>>>>>>>>>>>> PLANE 1 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not running registration, plane already registered
binary path: \\nimhlabstore1.nimh.nih.gov\UFNC\FNC3Z\Sutie2p-Processed\GCamP6S-CamKII\\SL088604152025\sub1-2Spon\suite2p\plane1\data.bin
NOTE: applying default C:\Users\zhangl33\.suite2p\classifiers\classifier_user.npy
----------- ROI DETECTION
Binning movie in chunks of length 10
Binned movie of size [820,508,508] created in 67.15 sec.
NOTE: estimated spatial scale ~6 pixels, time epoc

In [8]:
full_save_path

'\\\\nimhlabstore1.nimh.nih.gov\\UFNC\\FNC3Z\\Sutie2p-Processed\\GCamP6S-CamKII\\\\SL085503042025\\sub1-2Spon'

Simply combine previously processed plane data to combined data on server